This notebook gives an example of how one can analyze different runs in the same database. 
This is for instance relevant if multiple analysis with veiligheidsrendement are made for the same traject.

### Import necessary libraries

In [1]:
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import numpy as np
from scipy.stats import norm
from peewee import fn
from collections import defaultdict
from vrtool.orm.models import *
from vrtool.orm.orm_controllers import open_database
from vrtool.common.enums import MechanismEnum
from postprocessing.database_analytics import *
from postprocessing.database_access_functions import * 
from postprocessing.generate_output import *
from scipy.interpolate import interp1d

sns.set(style="whitegrid")
colors = sns.color_palette("colorblind", 10)



### Get the runs that are in the database
First we get an overview of the runs in the database

In [3]:
database_path = Path(r'c:\Users\rikkert\OneDrive - Stichting Deltares\Desktop\dec2024_backup_C\VRM\traject10_voor_handreiking\10-2\database_10-2.sqlite')
run_list = get_overview_of_runs(database_path)
run_list = [run for run in run_list if run['optimization_type_name']== 'VEILIGHEIDSRENDEMENT']
pd.DataFrame(run_list)
print(run_list)

[{'id': 1, 'name': 'Basisberekening Veiligheidsrendement', 'discount_rate': 0.03, 'optimization_type': 1, 'optimization_type_name': 'VEILIGHEIDSRENDEMENT'}]


For each run, we get the optimization steps and the step with minimal total cost

In [4]:
optimization_steps = {run['name']: get_optimization_steps_for_run_id(database_path, run['id']) for run in run_list}
# add total cost as sum of total_lcc and total_risk in each step

minimal_tc_steps = {run: get_minimal_tc_step(steps) for run, steps in optimization_steps.items()}


### Reading measures per step
The next step is to read the measures and parameters of these measures for each optimization step such that we can compare the measures that are taken in each step and for each section.

In [5]:
lists_of_measures = {run['id']: get_measures_for_run_id(database_path, run['id']) for run in run_list}

measures_per_step = {run['id']: get_measures_per_step_number(lists_of_measures[run['id']]) for run in run_list}

If we want to see the failure probability per stap we first need to load the original assessment for each mechanism, and then we can compute the reliability for each step during the optimization. 

In [6]:
assessment_results = {mechanism: import_original_assessment(database_path, mechanism) 
                      for mechanism in [MechanismEnum.OVERFLOW, MechanismEnum.PIPING, MechanismEnum.STABILITY_INNER]}

reliability_per_step = {run['id']: get_reliability_for_each_step(database_path, measures_per_step[run['id']]) for run in run_list}

Based on these inputs we can make a stepwise_assessment based on the investments in reliability_per_step

In [7]:
stepwise_assessment = {run['id']: assessment_for_each_step(copy.deepcopy(assessment_results), reliability_per_step[run['id']]) for run in run_list}

The next step is to derive the traject probability for each mechanism for each step using the `calculate_traject_probability_for_steps` function

In [8]:
traject_prob = {run['id']: calculate_traject_probability_for_steps(stepwise_assessment[run['id']]) for run in run_list}

for count, run in enumerate(run_list):
    print(traject_prob[run['id']][minimal_tc_steps[run['name']]])


{<MechanismEnum.OVERFLOW: 1>: {0: 2.244486981467377e-05, 100: 0.00013541632842700686, 75: 7.914169132520554e-05, 50: 5.177526375629467e-05, 19: 3.0972984132554564e-05, 20: 3.149771415155319e-05, 25: 3.4250030752859175e-05}, <MechanismEnum.PIPING: 3>: {0: 9.76256788269847e-05, 100: 0.0002750553105085318, 75: 0.0002066344877924564, 50: 0.00015658125625073183, 19: 0.00011488925773861514, 20: 0.00011595187478596891, 25: 0.0001215197598783968}, <MechanismEnum.STABILITY_INNER: 2>: {0: 2.4783444973697222e-05, 100: 2.4783444973697222e-05, 75: 2.4783444973697222e-05, 50: 2.4783444973697222e-05, 19: 2.4783444973697222e-05, 20: 2.4783444973697222e-05, 25: 2.4783444973697222e-05}}


Now we check the measures for each section. We print the ids of the measures

In [9]:
measures_per_section = {run['id']: get_measures_per_section_for_step(measures_per_step[run['id']], minimal_tc_steps[run['name']]) for run in run_list}
section_names = [list(measures_per_section[run].keys()) for run in measures_per_section.keys()]
section_names = list(set([item for sublist in section_names for item in sublist]))

for section in section_names:
    for run in measures_per_section.keys():
        try:
            print(f"Section {section} in run {run} has measures {measures_per_section[run][section][0]} at time {measures_per_section[run][section][1]}")  
        except:
            print(f"Section {section} in run {run} has no measures in run {run}")

Section 1 in run 1 has measures [217, 1] at time [0, 0]
Section 2 in run 1 has measures [438, 222] at time [0, 0]
Section 3 in run 1 has measures [659, 443] at time [0, 0]
Section 4 in run 1 has measures [880, 664] at time [0, 0]
Section 5 in run 1 has measures [1101, 885] at time [0, 0]
Section 6 in run 1 has measures [1322, 1106] at time [0, 0]
Section 7 in run 1 has measures [1543, 1327] at time [0, 0]
Section 11 in run 1 has measures [2427, 2211] at time [0, 0]
Section 12 in run 1 has measures [2648, 2432] at time [0, 0]
Section 15 in run 1 has measures [3311, 3095] at time [0, 0]
Section 20 in run 1 has measures [4205] at time [0]
Section 22 in run 1 has measures [4646] at time [0]
Section 24 in run 1 has measures [5089] at time [0]
Section 25 in run 1 has measures [5308] at time [0]
Section 26 in run 1 has measures [5532] at time [0]
Section 27 in run 1 has measures [5966] at time [0]
Section 28 in run 1 has measures [5972] at time [0]
Section 29 in run 1 has measures [6192] at t

Now we get for each section the parameters of the measure + timing + cost. This is stored in a `pd.DataFrame` for each run.

In [54]:
section_parameters = defaultdict(dict)

for run in measures_per_section.keys():
    for section in measures_per_section[run].keys():
        section_parameters[run][section] = []
        for measure in measures_per_section[run][section][0]:
            parameters = get_measure_parameters(measure, database_path)
            parameters.update(get_measure_costs(measure, database_path))
            parameters.update(get_measure_type(measure, database_path))
            # if parameters name is "Grondversterking binnenwaarts" and dberm and dcrest are 0, set cost to 0
            if parameters['name'] == 'Grondversterking binnenwaarts' and parameters['dberm'] == 0 and parameters['dcrest'] == 0:
                print(f"Setting costs to 0 for measure {parameters['name']} in section {section} in run {run}")
                parameters['cost'] = 0
            section_parameters[run][section].append(parameters)

measure_parameters = {run['id']: measure_per_section_to_df(measures_per_section[run['id']], section_parameters[run['id']]) for run in run_list}


Setting costs to 0 for measure Grondversterking binnenwaarts in section 1 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 2 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 3 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 4 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 5 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 6 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 7 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 11 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 12 in run 1
Setting costs to 0 for measure Grondversterking binnenwaarts in section 15 in run 1


We list the investment for each year for each section

In [55]:
# we need final step of stepwise_assessment
# and assessment_results for initial state of each section
# we also have costs per section.

# determine final beta for traject:
initial_traject_probability_per_mechanism = calculate_traject_probability(assessment_results)
print(f"Initial traject probability is {initial_traject_probability_per_mechanism}")

n_time_steps = len(initial_traject_probability_per_mechanism[MechanismEnum.OVERFLOW])
time_steps = initial_traject_probability_per_mechanism[MechanismEnum.OVERFLOW].keys()
print(f"Number of time steps is {n_time_steps}")
print(f"Time steps are {time_steps}")

# print final step of stepwise_assessment (at minimal_tc_steps):
for count, run in enumerate(run_list):
    final_traject_probability_per_mechanism = traject_prob[run['id']][minimal_tc_steps[run['name']]]
    final_section_probability_per_mechanism = stepwise_assessment[run['id']][minimal_tc_steps[run['name']]]

print(f"Final traject probability is {final_traject_probability_per_mechanism}")
print(f"Final section probability is {final_section_probability_per_mechanism}")

Initial traject probability is {<MechanismEnum.OVERFLOW: 1>: {0: 8.625642325360978e-05, 100: 0.0006495013452424325, 75: 0.0004023108390659708, 50: 0.0002449672824359944, 19: 0.00012929450525212818, 20: 0.00013204201587619678, 25: 0.0001466198539549501}, <MechanismEnum.PIPING: 3>: {0: 0.013009664252130904, 100: 0.02309309486654454, 75: 0.01988226167504592, 50: 0.017171891847211573, 19: 0.014412970462163277, 20: 0.014492556714083116, 25: 0.014899476451536886}, <MechanismEnum.STABILITY_INNER: 2>: {0: 0.00012919708498704274, 100: 0.00012919708498704274, 75: 0.00012919708498704274, 50: 0.00012919708498704274, 19: 0.00012919708498704274, 20: 0.00012919708498704274, 25: 0.00012919708498704274}}
Number of time steps is 7
Time steps are dict_keys([0, 100, 75, 50, 19, 20, 25])
Final traject probability is {<MechanismEnum.OVERFLOW: 1>: {0: 2.244486981467377e-05, 100: 0.00013541632842700686, 75: 7.914169132520554e-05, 50: 5.177526375629467e-05, 19: 3.0972984132554564e-05, 20: 3.149771415155319e-05

In [56]:
# read from the database the economic damage, which is found in DikeTrajectInfo, and called flood_damage
# there is only 1 value in total, so we can use the first value

with open_database(database_path) as db:
    damage = DikeTrajectInfo.select(DikeTrajectInfo.flood_damage).where(DikeTrajectInfo.id == 1).get().flood_damage

print(f"Damage is {damage}")

Damage is 700000000.0


In [57]:
discount_rate = 0.03

damage_per_year = np.divide(damage, np.power(1+discount_rate, np.arange(0,100)))
damage_per_year = damage_per_year.reshape(1,100)
print(damage_per_year)

[[7.00000000e+08 6.79611650e+08 6.59817136e+08 6.40599162e+08
  6.21940934e+08 6.03826149e+08 5.86238980e+08 5.69164058e+08
  5.52586464e+08 5.36491713e+08 5.20865740e+08 5.05694894e+08
  4.90965916e+08 4.76665938e+08 4.62782464e+08 4.49303363e+08
  4.36216857e+08 4.23511512e+08 4.11176225e+08 3.99200219e+08
  3.87573028e+08 3.76284493e+08 3.65324751e+08 3.54684224e+08
  3.44353615e+08 3.34323898e+08 3.24586309e+08 3.15132339e+08
  3.05953727e+08 2.97042454e+08 2.88390732e+08 2.79991002e+08
  2.71835924e+08 2.63918373e+08 2.56231430e+08 2.48768378e+08
  2.41522698e+08 2.34488056e+08 2.27658307e+08 2.21027482e+08
  2.14589789e+08 2.08339601e+08 2.02271457e+08 1.96380055e+08
  1.90660248e+08 1.85107037e+08 1.79715570e+08 1.74481135e+08
  1.69399161e+08 1.64465205e+08 1.59674956e+08 1.55024229e+08
  1.50508960e+08 1.46125204e+08 1.41869130e+08 1.37737020e+08
  1.33725262e+08 1.29830351e+08 1.26048885e+08 1.22377558e+08
  1.18813163e+08 1.15352585e+08 1.11992801e+08 1.08730875e+08
  1.0556

In [58]:
# function that calculates total risk, given traject reliability per mechanism, damage and annual discount rate

def calculate_total_risk(traject_reliability, damage, discount_rate):
    n_years = 100
    damage_per_year = np.divide(damage, np.power(1+discount_rate, np.arange(0,n_years)))
    damage_per_year = damage_per_year.reshape(1,n_years)
    total_non_failure_probability = np.ones([1,n_years])
    traject_reliability_interp = {}
    for key in traject_reliability.keys():
        times,betas = zip(*traject_reliability[key].items())
        time_beta_interpolation = interp1d(times, betas, kind='linear', fill_value='extrapolate')
        traject_reliability_interp[key] = time_beta_interpolation(list(range(0,100)))
        traject_reliability_interp[key] = np.array(traject_reliability_interp[key]).reshape(1,100)
    for key in traject_reliability_interp.keys():
        total_non_failure_probability = np.multiply(total_non_failure_probability, 1-traject_reliability_interp[key])
    total_failure_probability = 1 - total_non_failure_probability
    expected_risk_per_year = np.multiply(damage_per_year, total_failure_probability)
    total_risk = np.sum(expected_risk_per_year)
    print(f"Total risk is {int(total_risk)}")
    return total_risk

total_risk = calculate_total_risk(final_traject_probability_per_mechanism, damage, discount_rate)

Total risk is 4470095


In [59]:
# Backward VRM index calculation:
# section by section, replace the final traject probability by the initial traject probability of that section
# then recalculate the traject failure probability, and calculate the increase in risk. We find the VR-index by dividing the increase in risk by the costs of the measure

# create empty lists and dictionaries
increase_in_traject_risk = []
section_costs = []
vr_index = {}

for section in section_names:
    final_section_probability_per_mechanism_temp = copy.deepcopy(final_section_probability_per_mechanism)

    for mechanism in assessment_results.keys():
        final_section_probability_per_mechanism_temp[mechanism][section]['beta'] = assessment_results[mechanism][section]['beta']

    # recalculate final traject probability
    final_traject_probability_per_mechanism_temp = calculate_traject_probability(final_section_probability_per_mechanism_temp)
    
    # calculate_total_risk
    risk_increased = calculate_total_risk(final_traject_probability_per_mechanism_temp, damage, discount_rate) 
    delta_risk = risk_increased - total_risk

    if section in list(measure_parameters[1]['section_id']):
        if measure_parameters[1][(measure_parameters[1]['section_id'] == section) & (measure_parameters[1]['name'] == 'Grondversterking binnenwaarts') & (measure_parameters[1]['dcrest'] == 0) & (measure_parameters[1]['dberm'] == 0)].shape[0] > 0:
            print(f"Section {section} has grondversterking 0/0")
            vr_index[section] = 0
        else:
            section_costs = measure_parameters[1][measure_parameters[1]['section_id'] == section]['LCC'].values[0]
            vr_index[section] = delta_risk / section_costs
    else:
        vr_index[section] = 0



Total risk is 62892553
Total risk is 19221911
Total risk is 19117426
Total risk is 5894006
Total risk is 13923417
Total risk is 167868849
Total risk is 5719050
Total risk is 16618752
Total risk is 9393161
Total risk is 60490952
Total risk is 4920108
Total risk is 4825026
Total risk is 5194723
Total risk is 4857436
Total risk is 4981558
Total risk is 5672328
Total risk is 5307179
Total risk is 5205498
Total risk is 5433650
Total risk is 5658297
Total risk is 5490366
Total risk is 7826263
Total risk is 8384354
Total risk is 8511191
Total risk is 5524206
Total risk is 5130507
Total risk is 5051418
Total risk is 5259234
Total risk is 4647860
Total risk is 4643292


In [60]:
measure_parameters[1]

,section_id,name,LCC,dcrest,dberm,beta_target,transition_level,investment_years,l_stab_screen
0,1,Verticale pipingoplossing + Grondversterking b...,5.270000e+05,0.00,0.0,NaN,NaN,0 + 0,NaN
1,2,Verticale pipingoplossing + Grondversterking b...,8.670000e+05,0.00,0.0,NaN,NaN,0 + 0,NaN
2,3,Verticale pipingoplossing + Grondversterking b...,1.190000e+05,0.00,0.0,NaN,NaN,0 + 0,NaN
3,4,Verticale pipingoplossing + Grondversterking b...,1.037000e+06,0.00,0.0,NaN,NaN,0 + 0,NaN
4,5,Verticale pipingoplossing + Grondversterking b...,4.590000e+05,0.00,0.0,NaN,NaN,0 + 0,NaN
5,6,Verticale pipingoplossing + Grondversterking b...,2.380000e+05,0.00,0.0,NaN,NaN,0 + 0,NaN
6,7,Verticale pipingoplossing + Grondversterking b...,8.330000e+05,0.00,0.0,NaN,NaN,0 + 0,NaN
7,11,Verticale pipingoplossing + Grondversterking b...,4.420000e+05,0.00,0.0,NaN,NaN,0 + 0,NaN
8,12,Verticale pipingoplossing + Grondversterking b...,3.400000e+05,0.00,0.0,NaN,NaN,0 + 0,NaN
9,15,Verticale pipingoplossing + Grondversterking b...,1.037000e+06,0.00,0.0,NaN,NaN,0 + 0,NaN


In [36]:
print(vr_index)

# sort the dictionary by value
sorted_vr_index = dict(sorted(vr_index.items(), key=lambda item: item[1], reverse=True))
print(sorted_vr_index)


{1: 96.52513836552566, 2: 14.8148625717308, 3: 107.17235181597123, 4: 1.195570708918989, 5: 17.932589614768027, 6: 597.782239762416, 7: 1.3054882103604804, 11: 23.931903050340264, 12: 12.60747098596051, 15: 47.037281470267516, 20: 2.624401432870569, 22: 2.3510380963665205, 24: 6.043631019916744, 25: 1.8764808825452868, 26: 10.565431535385533, 27: 4.5256286958768355, 28: 5.593942127629342, 29: 2.7111458977246987, 30: 2.6997151961980355, 31: 23.38943099421208, 32: 3.8886436054115507, 33: 8.647475558803606, 34: 8.745394003080758, 35: 6.575926511495278, 36: 9.05689842107273, 37: 3.0764361557780604, 38: 1.6476622590219185, 39: 11.055148181866016, 40: 1.0580776408579848, 41: 2.945381108254439}
{6: 597.782239762416, 3: 107.17235181597123, 1: 96.52513836552566, 15: 47.037281470267516, 11: 23.931903050340264, 31: 23.38943099421208, 5: 17.932589614768027, 2: 14.8148625717308, 12: 12.60747098596051, 39: 11.055148181866016, 26: 10.565431535385533, 36: 9.05689842107273, 34: 8.745394003080758, 33: 8

In [18]:
def get_forward_vr_order(measures_per_step):
        forward_vr_order = [step['section_id'][0] for _idx, (step) in enumerate(measures_per_step[run['id']].values()) if _idx <= minimal_tc_steps[run['name']]]
        #take first of unique values, keep order
        forward_vr_order = [x for i, x in enumerate(forward_vr_order) if forward_vr_order.index(x) == i]
        return forward_vr_order

order_forward_vr = get_forward_vr_order(measures_per_step)

print(order_forward_vr)

[6, 3, 1, 2, 15, 31, 11, 5, 39, 36, 26, 33, 12, 27, 28, 24, 32, 34, 35, 37, 29, 30, 20, 41, 22, 25, 38, 7, 4, 40]


In [29]:
section_reliability_assessment_list = get_section_assessment_results(database_path)
print(section_reliability_assessment_list)

# get beta for each section at 'time'
time = 25

section_reliability_assessment_list = [section for section in section_reliability_assessment_list if section['time'] == time]
print(section_reliability_assessment_list)

for section_data in section_reliability_assessment_list:
    section_data['Pf'] = norm.cdf(-section_data['beta'])

# sort the list based on Pf and print sorted list in descending order
section_reliability_assessment_list_sorted_pf = sorted(section_reliability_assessment_list, key=lambda k: k['Pf'], reverse=True)
print(section_reliability_assessment_list_sorted_pf)

# print section ids and Pf
print()
for section in section_reliability_assessment_list_sorted_pf:
    print(f"section_id: {section['section_data']}, Pf: {section['Pf']}")


[{'id': 1, 'beta': 2.891447807517554, 'time': 0, 'section_data': 1}, {'id': 2, 'beta': 2.832174821650513, 'time': 19, 'section_data': 1}, {'id': 3, 'beta': 2.8290955519392957, 'time': 20, 'section_data': 1}, {'id': 4, 'beta': 2.8137579548653977, 'time': 25, 'section_data': 1}, {'id': 5, 'beta': 2.7384850606334563, 'time': 50, 'section_data': 1}, {'id': 6, 'beta': 2.665417571736322, 'time': 75, 'section_data': 1}, {'id': 7, 'beta': 2.5943574024790603, 'time': 100, 'section_data': 1}, {'id': 8, 'beta': 3.3024588096382645, 'time': 0, 'section_data': 2}, {'id': 9, 'beta': 3.2474784341460765, 'time': 19, 'section_data': 2}, {'id': 10, 'beta': 3.244610838250576, 'time': 20, 'section_data': 2}, {'id': 11, 'beta': 3.23031140106975, 'time': 25, 'section_data': 2}, {'id': 12, 'beta': 3.15975847263553, 'time': 50, 'section_data': 2}, {'id': 13, 'beta': 3.0907219504639665, 'time': 75, 'section_data': 2}, {'id': 14, 'beta': 3.0231229602934526, 'time': 100, 'section_data': 2}, {'id': 15, 'beta': 3.3

In [28]:
# create a pandas dataframe with all sections in database that are inanalyse = True

def get_sections_in_analysis(database_path):
    with open_database(database_path) as db:
        sections = SectionData.select().where(SectionData.in_analysis == True)
        sections_analysis = pd.DataFrame()
        # add section.section_name to the dataframe, using pandas concat
        for section in sections:
            sections_analysis = pd.concat([sections_analysis, pd.DataFrame({'section_name': [section.section_name]})], ignore_index=True)
        # section names are sometimes integers, sometimes strings. Try to make them integers if possible
        try:
            sections_analysis['section_name'] = sections_analysis['section_name'].astype(int)
        except:
            pass
    return sections_analysis

sections_analysis = get_sections_in_analysis(database_path)

print(sections_analysis)
print(sections_analysis.dtypes)

    section_name
0              1
1              2
2              3
3              4
4              5
5              6
6              7
7              8
8              9
9             10
10            11
11            12
12            13
13            14
14            15
15            16
16            17
17            18
18            19
19            20
20            21
21            22
22            23
23            24
24            25
25            26
26            27
27            28
28            29
29            30
30            31
31            32
32            33
33            34
34            35
35            36
36            37
37            38
38            39
39            40
40            41
41            42
42            43
43            44
section_name    int32
dtype: object


In [74]:
# now we want to expand the dataframe. First we want to add the initial failure probability of a section at t = 25, as it resembles the situation in 2050

t = 25 # we set the time to 25 years from now to have the failure probability in 2050

for section_name in sections_analysis.section_name:
    # failure probability of a section is the sum of the failure probabilities of the mechanisms
    initial_section_failure_prob = 0
    for mechanism in assessment_results.keys():
        times,betas = zip(*assessment_results[mechanism][section_name].items())
        print(times, betas) 
        # time contains 0, 19, 20, 25, 50, 75, 100        # find beta where time == t (=25)
        beta = betas[times.index(t)]
    sections_analysis.iloc[section_name, 'initial_failure_prob'] = initial_section_failure_prob



('time', 'beta') ([0, 19, 20, 25, 50, 75, 100], [4.553393506493507, 4.485881818181818, 4.482328571428572, 4.46456233766234, 4.3757311688311695, 4.2869, 4.198068831168833])


ValueError: tuple.index(x): x not in tuple